In [1]:
import pyvistaqt as pvqt

In [2]:
try:
    pl.close()
except NameError:
    pass
pl = pvqt.BackgroundPlotter()

In [3]:
import pyvista as pv
import h5py

In [4]:
from dolfin import *

In [5]:
def compute_vectorfields(p: Function):
    V = p.function_space()
    mesh = V.mesh()
    degree = V.ufl_element().degree()
    W = VectorFunctionSpace(mesh, "P", degree)
    return [project(-grad(pi), W) for pi in p]

def compute_vectorfield(p: Function):
    V = p.function_space()
    mesh = V.mesh()
    degree = V.ufl_element().degree()
    W = VectorFunctionSpace(mesh, "P", degree)
    return project(-grad(p), W)

In [6]:
mesh = UnitSquareMesh(10, 10)
c = (0, 0, 0)
l = 1.

u0 = Expression("exp(-((pow(x[0]-c[0], 2) + pow(x[1]-c[1], 2))/ (l * l)))", c=Constant(c), l=Constant(l), degree=2)
V = FunctionSpace(mesh, "CG", 1)
u = project(u0, V)
u.rename("pressure", "p")

# v = compute_vectorfield(u)
# v.rename("velocity", "v")

v = Function(V)
v = project(-u, V)
v.rename("vel_mag", "v")

with XDMFFile("testfile.xdmf") as f:
    f.write(u, 0.0)
    f.write(v, 0.0)

In [6]:
import meshio
import numpy as np
import vtk

In [26]:
pvmesh

NameError: name 'pvmesh' is not defined

In [61]:
reader = meshio.xdmf.TimeSeriesReader("testfile.xdmf")


times = []
point_dataset = []
cell_data_set = []
points, cells = reader.read_points_cells()
points = np.c_[ points, np.zeros(points.shape[0])]

In [37]:
for c in cells:
    print(c)

<meshio CellBlock, type: triangle, num cells: 200, tags: []>


In [ ]:
N = cells[0].data.shape[0]

In [72]:
points.shape

(121, 3)

In [73]:
cells[0].data.shape

(200, 3)

In [74]:
N

200

In [71]:
pv.UnstructuredGrid(cells[0].data, [pv.CellType.TRIANGLE] * N)

ValueError: Number of cell types (200) must match the number of cells 21)

In [38]:
points

array([[0. , 0. ],
       [0.1, 0. ],
       [0.2, 0. ],
       [0.3, 0. ],
       [0.4, 0. ],
       [0.5, 0. ],
       [0.6, 0. ],
       [0.7, 0. ],
       [0.8, 0. ],
       [0.9, 0. ],
       [1. , 0. ],
       [0. , 0.1],
       [0.1, 0.1],
       [0.2, 0.1],
       [0.3, 0.1],
       [0.4, 0.1],
       [0.5, 0.1],
       [0.6, 0.1],
       [0.7, 0.1],
       [0.8, 0.1],
       [0.9, 0.1],
       [1. , 0.1],
       [0. , 0.2],
       [0.1, 0.2],
       [0.2, 0.2],
       [0.3, 0.2],
       [0.4, 0.2],
       [0.5, 0.2],
       [0.6, 0.2],
       [0.7, 0.2],
       [0.8, 0.2],
       [0.9, 0.2],
       [1. , 0.2],
       [0. , 0.3],
       [0.1, 0.3],
       [0.2, 0.3],
       [0.3, 0.3],
       [0.4, 0.3],
       [0.5, 0.3],
       [0.6, 0.3],
       [0.7, 0.3],
       [0.8, 0.3],
       [0.9, 0.3],
       [1. , 0.3],
       [0. , 0.4],
       [0.1, 0.4],
       [0.2, 0.4],
       [0.3, 0.4],
       [0.4, 0.4],
       [0.5, 0.4],
       [0.6, 0.4],
       [0.7, 0.4],
       [0.8,

In [39]:
cells[0].data

array([[  0,   1,  12],
       [  0,  11,  12],
       [  1,   2,  13],
       [  1,  12,  13],
       [  2,   3,  14],
       [  2,  13,  14],
       [  3,   4,  15],
       [  3,  14,  15],
       [  4,   5,  16],
       [  4,  15,  16],
       [  5,   6,  17],
       [  5,  16,  17],
       [  6,   7,  18],
       [  6,  17,  18],
       [  7,   8,  19],
       [  7,  18,  19],
       [  8,   9,  20],
       [  8,  19,  20],
       [  9,  10,  21],
       [  9,  20,  21],
       [ 11,  12,  23],
       [ 11,  22,  23],
       [ 12,  13,  24],
       [ 12,  23,  24],
       [ 13,  14,  25],
       [ 13,  24,  25],
       [ 14,  15,  26],
       [ 14,  25,  26],
       [ 15,  16,  27],
       [ 15,  26,  27],
       [ 16,  17,  28],
       [ 16,  27,  28],
       [ 17,  18,  29],
       [ 17,  28,  29],
       [ 18,  19,  30],
       [ 18,  29,  30],
       [ 19,  20,  31],
       [ 19,  30,  31],
       [ 20,  21,  32],
       [ 20,  31,  32],
       [ 22,  23,  34],
       [ 22,  33

In [41]:
?pv.UnstructuredGrid

In [45]:
pyviz = pv.UnstructuredGrid(points)
pyviz

TypeError: Cannot work with input type <class 'numpy.ndarray'>

In [ ]:
def read_xdmf_timeseries(filename, idx=None, variables=None):
    times = []
    point_dataset = []
    cell_data_set = []
    with meshio.xdmf.TimeSeriesReader(filename) as reader:
        points, cells = reader.read_points_cells()
        if points.shape[1]==2:
            points = np.append(points, np.zeros((points.shape[0], 1)), axis=-1)
        if idx is None:
            idx = range(reader.num_steps)
        for k in idx:
            t, point_data, cell_data = reader.read_data(k)
            times.append(t)
            data_dict = dict()
            for name, data in point_data.items():
                if variables is not None:
                    if name in variables:
                        data_dict[name] = data
                else:
                        data_dict[name] = data
            point_dataset.append(data_dict)
            data_dict = dict()
            for name, data in cell_data.items():
                if variables is not None:
                    if name in variables:
                        data_dict[name] = data
                else:
                        data_dict[name] = data
            cell_data_set.append(data_dict)
            
    return times, points, cells, point_dataset, cell_data_set


def xdmf_to_unstructuredgrid(filename, idx=None, variables=None):
    times, points, cells, point_dataset, cell_dataset = read_xdmf_timeseries(filename, idx=idx,
                                                                             variables=variables)
    n = cells[0].data.shape[0] # number of cells
    p = cells[0].data.shape[1] # number of points per cell
    c = cells[0].data
    
    if (points[:,2]==0).all():
        cell_type = vtk.VTK_TRIANGLE
    else:
        cell_type = vtk.VTK_TETRA

    c = np.insert(c, 0, p, axis=1) # insert number of points per cell at begin of cell row
    grid = pv.UnstructuredGrid(c, np.repeat(cell_type, n), points)
    for i, p_data in enumerate(point_dataset):   
        if len(point_dataset)==1:
            array_name = "{name}"  
        else:
            array_name = "{name}_{idx}"
        for name, data in p_data.items():
            grid.point_data[array_name.format(name=name, idx=i)] = data
    for i, c_data in enumerate(cell_dataset):
        for name, data in c_data.items():
            grid.cell_arrays[array_name.format(name=name, idx=i)] = data
    return grid

In [42]:
try:
    pl.close()
except NameError:
    pass

pl = pvqt.BackgroundPlotter()
pl.clear()
pvmesh = xdmf_to_unstructuredgrid("testfile.xdmf") 
pl.add_mesh(pvmesh)
pvmesh["data"] = -u.vector()[:] # Set new value (need to find reordering)

NameError: name 'xdmf_to_unstructuredgrid' is not defined